# XGBoost Model Hypertuning

In [1]:
# Load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
import csv
import tensorflow as tf

from xgboost import XGBRegressor
import datetime
import seaborn as sns

In [2]:
# Set random seed for TensorFlow
tf.random.set_seed(123)

# Set random seed for Python
np.random.seed(123)

tf.keras.utils.set_random_seed(1)

tf.config.experimental.enable_op_determinism()

In [3]:
# Load the data
data_all_county = pd.read_csv('data/CA_data_lat_log_weekly.csv')

In [4]:
# Convert the date to datetime to extract the month value
data_all_county['date'] = pd.to_datetime(data_all_county['date'])
data_all_county['month'] = data_all_county['date'].dt.month
data_all_county['month'] = data_all_county['month'].astype('category')

# Processing Functions

In [5]:
def ts_multi_data_prep(dataset, target, start, end, window, step_out):
    '''Slices the dataset into sliding and overlapping windows of desired window size and forecast horizon'''
    
    X = []
    y = []
    start = start + window
    if end is None:
        end = len(dataset) - step_out
        #end = len(dataset)
    for i in range(start, end):
        indices = range(i-window, i)
        X.append(dataset[indices])

        indicey = range(i, i+step_out) #revise the window definition
        y.append(target[indicey])
    return np.array(X), np.array(y)

In [6]:
def timeseries_evaluation_metrics_func(y_true, y_pred):
    '''Calculates and returns the evaluation metrics, MSE and MAE'''

    print('Evaluation metric results:-')
    mse = metrics.mean_squared_error(y_true.flatten(), y_pred.flatten())
    mae = metrics.mean_absolute_error(y_true.flatten(), y_pred.flatten())
    
    # Can calculate RMSE and R2 and print results if desired
    #rmse = np.sqrt(mse)
    #r2 = metrics.r2_score(y_true.flatten(), y_pred.flatten())
    #print(f'MSE is : {mse}')
    #print(f'MAE is : {mae}')
    #print(f'RMSE is : {rmse}')
    #print(f'R2 is : {r2}\n')
    return mse, mae

In [7]:
def timeseries_evaluation_metrics_binary(y_true, y_pred):
    '''Calculates and returns the F1 score for the binary classification task'''
    
    print('Evaluation metric results:-')
    accuracy = accuracy_score(y_true.flatten(), y_pred.flatten())
    precision = precision_score(y_true.flatten(), y_pred.flatten(), average='macro')
    recall = recall_score(y_true.flatten(), y_pred.flatten(), average='macro')
    f1 = f1_score(y_true.flatten(), y_pred.flatten(), average='macro')

    #print(f'Accuracy: {accuracy}')
    #print(f'Precision: {precision}')
    #print(f'Recall: {recall}')
    #print(f'F1-score: {f1}\n')
    return f1

In [8]:
def transform_county_data(x_data_array, y_data_array):
    # Lists to store x_train_c and y_train_c arrays
    x_c_list = []
    y_c_list = []
    
    # Divide the arrays into 'unique_fips_count' number of subarrays
    x_subarrays = np.array_split(x_data_array, unique_fips_count, axis=0)
    y_subarrays = np.array_split(y_data_array, unique_fips_count, axis=0)

    # Combine x_subarrays and y_subarrays into tuples
    data_tuples = [(x_subarray, y_subarray) for x_subarray, y_subarray in zip(x_subarrays, y_subarrays)]

    # Print or use the data tuples as needed
    for idx, data_tuple in enumerate(data_tuples):
        x_window_c, y_window_c = ts_multi_data_prep(data_tuple[0],data_tuple[1], 0, None, hist_window, step_out)
        # Append x_window_c and y_window_c arrays to lists
        x_c_list.append(x_window_c)
        y_c_list.append(y_window_c)

    # Stack arrays in lists to create x_train_c and y_train_c
    x_all_county = np.vstack(x_c_list)
    y_all_county = np.vstack(y_c_list)

    return x_all_county, y_all_county

# Data Processing

In [9]:
# Create empty lists to hold arrays
x_train_c, y_train_c, x_vali_c, y_vali_c, x_test_c, y_test_c = [], [], [], [], [], []

# Get the unique FIPS or county codes
unique_fips = data_all_county['fips'].unique()
unique_fips_count = data_all_county['fips'].nunique()

for fips in unique_fips:
    # Extract dataframe for the current FIPS value
    data_county = data_all_county[data_all_county['fips'] == fips]

    # Select features to use in the dataset
    X_data = data_county[['lat','lon','PRECTOT', 'PS', 'QV2M', 'T2M', 'T2MDEW', 'T2MWET',
       'T2M_MAX', 'T2M_MIN', 'T2M_RANGE', 'TS', 'WS10M', 'WS10M_MAX',
       'WS10M_MIN', 'WS10M_RANGE', 'WS50M', 'WS50M_MAX', 'WS50M_MIN',
       'WS50M_RANGE', 'score', 'month']]
    
    # Set the labels
    Y_data = data_county[['score']]
    
    #train_val_test split 70%-10%-20%
    n = len(X_data)

    x_train_county = X_data[0:int(n*0.7)]
    y_train_county = Y_data[0:int(n*0.7)]
    x_vali_county = X_data[int(n*0.7):int(n*0.8)]
    y_vali_county = Y_data[int(n*0.7):int(n*0.8)]
    x_test_county = X_data[int(n*0.8):]
    y_test_county = Y_data[int(n*0.8):]

    # Condition for the first county in CA to start populating the lists
    if fips == 6001:
        x_train_c, y_train_c, x_vali_c, y_vali_c, x_test_c, y_test_c = x_train_county, y_train_county, x_vali_county, y_vali_county, x_test_county, y_test_county

    # Append the following counties to the lists
    else:
        x_train_c = np.concatenate((x_train_c, x_train_county), axis=0)
        y_train_c = np.concatenate((y_train_c, y_train_county), axis=0)
        x_vali_c = np.concatenate((x_vali_c, x_vali_county), axis=0)
        y_vali_c = np.concatenate((y_vali_c, y_vali_county), axis=0)
        x_test_c = np.concatenate((x_test_c, x_test_county), axis=0)
        y_test_c = np.concatenate((y_test_c, y_test_county), axis=0)


In [10]:
# Normalize the data
X_scaler_train = MinMaxScaler()
Y_scaler_train = MinMaxScaler()
X_scaler_test = MinMaxScaler()
Y_scaler_test = MinMaxScaler()
X_scaler_vali = MinMaxScaler()
Y_scaler_vali = MinMaxScaler()
x_train_data = X_scaler_train.fit_transform(x_train_c)
y_train_data = Y_scaler_train.fit_transform(y_train_c)
x_vali_data = X_scaler_vali.fit_transform(x_vali_c)
y_vali_data = Y_scaler_vali.fit_transform(y_vali_c)
x_test_data = X_scaler_test.fit_transform(x_test_c)
y_test_data = Y_scaler_test.fit_transform(y_test_c)


# Modeling

In [11]:
def modeling(hist_window, 
              step_out, 
              num_estimators, 
              max_depth, 
              learning_rate, 
              subsample,
              flag_report):
  '''Fits models for the different desired hyperparameters and returns results
  
  Hyperparameters
  
  Data size hyperparamters:
  hist_window: historical window size in number of weeks,
  step_out: forecast horizon size in number of weeks,
  
  XGBoost-specific hyperparameters (https://xgboost.readthedocs.io/en/stable/parameter.html):
  num_estimators
  max_depth
  learning_rate
  subsample
  '''

  x_train, y_train = transform_county_data(x_train_data, y_train_data)
  x_vali, y_vali = transform_county_data(x_vali_data, y_vali_data)
  x_test, y_test = transform_county_data(x_test_data, y_test_data)

  # Convert all the 3D arrays to 2D for XGBoost
  
  train_len = len(x_train)
  num_features = X_data.shape[1]
  vali_len = len(x_vali)
  test_len = len(x_test)

  # Reshape the labels into a simple 2D array
  y_train = y_train.reshape(train_len, step_out)
  y_vali = y_vali.reshape(vali_len, step_out)
  y_test = y_test.reshape(test_len, step_out)

  # Reshape the x data into a 2D array of (num windows, window size x features size)
  x_train = x_train.reshape(train_len, hist_window * num_features)
  x_vali = x_vali.reshape(vali_len, hist_window * num_features)
  x_test = x_test.reshape(test_len, hist_window * num_features)

  # fit the model
  model = XGBRegressor(objective='reg:squarederror',
                         n_estimators=num_estimators,
                         max_depth = max_depth,
                         learning_rate = learning_rate,
                         subsample = subsample)
  model.fit(x_train, y_train)

  # Get predictions
  y_test_pred = model.predict(x_test)
    
  # Convert predictions back onto real scale
  y_test_pred_Inverse = Y_scaler_test.inverse_transform(y_test_pred)
  y_test_pred_Inverse_ordinal = np.round(y_test_pred_Inverse).astype(int)
  y_test_Inverse = Y_scaler_test.inverse_transform(y_test)
  y_test_Inverse_ordinal = np.round(y_test_Inverse).astype(int)
  
  # Get evaluation mtrics
  mse, mae = timeseries_evaluation_metrics_func(y_test_Inverse,y_test_pred_Inverse)

  # Get the binary classification evaluation metrics
  threshold = 2.5
  y_test_Inverse_binary = np.where(y_test_Inverse >= threshold, 1, 0)
  y_test_pred_Inverse_binary = np.where(y_test_pred_Inverse >= threshold, 1, 0)
  f1 = timeseries_evaluation_metrics_binary(y_test_Inverse_binary,y_test_pred_Inverse_binary)
  
  # Print the classification report
  if flag_report:
    classification_metrics = classification_report(y_test_Inverse_binary.flatten(),y_test_pred_Inverse_binary.flatten())
    print(classification_metrics)

  print(f'Number of estimators: {num_estimators}, Max depth: {max_depth}, Learning rate: {learning_rate}, Subsample: {subsample}, F1: {f1}, MSE: {mse}, MAE: {mae}')
        
  return f1, mse, mae

## Window and Horizon Testing

In [15]:
parameter_result_list_window = []

# Set the desired XGBoost hyperparameters
num_estimators = 100
max_depth = 3
learning_rate = 0.15
subsample = 1

# Set initial values for window and horizon sizes
hist_window = 30
step_out = 12

# Loop through desired window and horizon sizes to compare results
for hist_window in [12, 24, 30, 40, 52]:
    for step_out in [4, 8, 12, 16]:
        f1, mse, mae = modeling(hist_window, 
                             step_out, 
                             num_estimators = num_estimators, 
                             max_depth = max_depth, 
                             learning_rate = learning_rate, 
                             subsample = 1, 
                             flag_report = 1)
        parameter_result_list_window.append((hist_window, 
                                   step_out, 
                                   num_estimators,
                                   max_depth,
                                   learning_rate,
                                   subsample, 
                                   f1, mse, mae))


# Printing the list with comment lines indicating parameter titles
print("# Hist_Window   Step_Out   Num Estimators  Max Depth   Learning Rate   Subsample   F1    MSE       MAE")

for params in parameter_result_list_window:
    print("{:<13} {:<10} {:<7} {:<7} {:<10} {:<10} {:<10.4f} {:<10.4f} {:<10.4f}".format(*params))


Evaluation metric results:-
Evaluation metric results:-
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     42235
           1       0.95      0.87      0.90      5093

    accuracy                           0.98     47328
   macro avg       0.96      0.93      0.95     47328
weighted avg       0.98      0.98      0.98     47328

Number of estimators: 100, Max depth: 3, Learning rate: 0.15, Subsample: 1, F1: 0.9462788459378368, MSE: 0.11963473901034674, MAE: 0.19024876113492895
Evaluation metric results:-
Evaluation metric results:-
              precision    recall  f1-score   support

           0       0.98      0.99      0.98     83346
           1       0.91      0.79      0.84      9454

    accuracy                           0.97     92800
   macro avg       0.94      0.89      0.91     92800
weighted avg       0.97      0.97      0.97     92800

Number of estimators: 100, Max depth: 3, Learning rate: 0.15, Subsample: 1, F1: 0.9

Evaluation metric results:-
Evaluation metric results:-
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     33130
           1       0.96      0.87      0.91      4918

    accuracy                           0.98     38048
   macro avg       0.97      0.93      0.95     38048
weighted avg       0.98      0.98      0.98     38048

Number of estimators: 100, Max depth: 3, Learning rate: 0.15, Subsample: 1, F1: 0.9505621398063535, MSE: 0.13103109724576678, MAE: 0.21603762607931296
Evaluation metric results:-
Evaluation metric results:-
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     64992
           1       0.93      0.79      0.85      9248

    accuracy                           0.97     74240
   macro avg       0.95      0.89      0.92     74240
weighted avg       0.96      0.97      0.96     74240

Number of estimators: 100, Max depth: 3, Learning rate: 0.15, Subsample: 1, F1: 0.9

In [16]:
# Save the parameter_result_list to a CSV file
with open('data/xgboost_parameter_result_list_windows.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(parameter_result_list)

In [17]:
# Load the parameter_result_list from the CSV file
parameter_result_list = []

with open('data/xgboost_parameter_result_list_windows.csv', 'r', newline='') as file:
    reader = csv.reader(file)
    for row in reader:
        parameter_result_list.append(row)
#parameter_result_list

In [18]:
# Convert all elements in the list to float
my_list_float = [[float(val) if '.' in val else int(val) for val in sublist] for sublist in parameter_result_list]

# Sort the list based on the f1 values
sorted_list = sorted(my_list_float, key=lambda x: x[6], reverse=True)

# Print the sorted list
for sublist in sorted_list:
    print(sublist)

[30, 4, 100, 3, 0.15, 1, 0.9522481063957986, 0.11713551367781273, 0.19334002834694375]
[40, 4, 100, 3, 0.15, 1, 0.9519247506737096, 0.12409801680637325, 0.20278282904570416]
[24, 4, 100, 3, 0.15, 1, 0.951837600480699, 0.11209263736576469, 0.18481798965605795]
[52, 4, 100, 3, 0.15, 1, 0.9505621398063535, 0.13103109724576678, 0.21603762607931296]
[12, 4, 100, 3, 0.15, 1, 0.9462788459378368, 0.11963473901034674, 0.19024876113492895]
[40, 8, 100, 3, 0.15, 1, 0.9176186002049829, 0.22173729772208944, 0.2966628010965734]
[52, 8, 100, 3, 0.15, 1, 0.9159672055598698, 0.23228815059856325, 0.31391418720822206]
[24, 8, 100, 3, 0.15, 1, 0.9155806911852923, 0.19760611870478384, 0.2698320964064132]
[30, 8, 100, 3, 0.15, 1, 0.9149312392328789, 0.20596715650900713, 0.2783315632975677]
[12, 8, 100, 3, 0.15, 1, 0.9130429332596823, 0.20701634843496627, 0.2721204877408845]
[30, 12, 100, 3, 0.15, 1, 0.8798821947216257, 0.29648042698318855, 0.34998273420429293]
[12, 12, 100, 3, 0.15, 1, 0.8793077719430614, 0